روی دیتای

cardio_train1

شبکه عمیق خود را به صورت های

sequentional functional classbase

و تیون کردن هایپر پارمتر ها ترین کنید

1- حتما تنسور بورد داشته باشد و به صورت یک لینک داده شود

2-EarlyStopping

3-  در ابتدای هر ایپاک شماره ایپاک و مقدار لاس را در متن زیر پرینت کند

in ... epoch loss is ...


In [1]:
%pip install -q -U tensorboard-plugin-profile

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pathlib import Path
from time import strftime
import tensorflow as tf 

2023-03-04 08:25:46.681744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 08:25:46.681834: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-04 08:26:19.494085: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-04 08:26:19.494702: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-04 08:26:19.494769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [19]:
import keras_tuner as kt

In [9]:
def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() # e.g.,

In [10]:
run_logdir

PosixPath('my_logs/run_2023_03_04_08_26_45')

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir,profile_batch=(100, 200))
history = model.fit([...], callbacks=[tensorboard_cb])

In [12]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

In [16]:
test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(str(test_logdir))
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10),step=step)
        data = (np.random.randn(100) + 2) * step / 100
        # gets larger
        tf.summary.histogram("my_hist", data, buckets=50,step=step)
        images = np.random.rand(2, 32, 32, 3) * step / 1000
        #gets brighter
        tf.summary.image("my_images", images, step=step)
        
        texts = ["The step is " + str(step), "Its square is " + str(step ** 2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000,step=step)

In [20]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8,default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4,max_value=1e-2,sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons,activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"])
    return model

In [26]:
df= pd.read_csv('cardio_train1.csv',index_col='id')

In [27]:
df

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
99996,19066,2,183,105.0,180,90,3,1,0,1,0,1


In [30]:
scaler = StandardScaler()

In [ ]:
X= df.iloc[:,:-1].values
Y=df.iloc[:,-1].values
X=scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=42)
X_val, X_test, y_val, y_test = train_test_split( X_test, y_test, test_size=0.33, random_state=42)

In [32]:
random_search_tuner = kt.RandomSearch(
build_model, objective="val_accuracy", max_trials=5,overwrite=True,directory="my_fashion_mnist", project_name="my_rnd_search",seed=42)
random_search_tuner.search(X_train, y_train, epochs=10,
validation_data=(X_val, y_val))

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.6403695940971375

Best val_accuracy So Far: 0.7377398610115051
Total elapsed time: 00h 03m 47s
INFO:tensorflow:Oracle triggered exit


In [33]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [34]:
top3_params =random_search_tuner.get_best_hyperparameters(num_trials=3)

In [41]:
top3_params[0].values

{'n_hidden': 4,
 'n_neurons': 74,
 'learning_rate': 0.00905127409782462,
 'optimizer': 'adam'}

In [42]:
best_trial =random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial summary
Hyperparameters:
n_hidden: 4
n_neurons: 74
learning_rate: 0.00905127409782462
optimizer: adam
Score: 0.7377398610115051


In [43]:
best_trial.metrics.get_last_value("val_accuracy")

0.7377398610115051

In [45]:
best_model.fit(X_train, y_train, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

Epoch 1/10
1466/1466 [==============================] - 5s 3ms/step - loss: 0.5519 - accuracy: 0.7280
Epoch 2/10
1466/1466 [==============================] - 3s 2ms/step - loss: 0.5512 - accuracy: 0.7290
Epoch 3/10
1466/1466 [==============================] - 4s 2ms/step - loss: 0.5505 - accuracy: 0.7277
Epoch 4/10
1466/1466 [==============================] - 3s 2ms/step - loss: 0.5503 - accuracy: 0.7295
Epoch 5/10
1466/1466 [==============================] - 4s 3ms/step - loss: 0.5489 - accuracy: 0.7278
Epoch 6/10
1466/1466 [==============================] - 4s 2ms/step - loss: 0.5497 - accuracy: 0.7306
Epoch 7/10
1466/1466 [==============================] - 3s 2ms/step - loss: 0.5734 - accuracy: 0.7302
Epoch 8/10
1466/1466 [==============================] - 3s 2ms/step - loss: 0.5489 - accuracy: 0.7289
Epoch 9/10
1466/1466 [==============================] - 4s 2ms/step - loss: 0.5487 - accuracy: 0.7300
Epoch 10/10
239/239 [==============================] - 1s 2ms/step - loss: 0.5443 

In [46]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):return build_model(hp)
    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [47]:
hyperband_tuner = kt.Hyperband(MyClassificationHyperModel(), objective="val_accuracy",seed=42,max_epochs=10, factor=3, hyperband_iterations=2,
overwrite=True, directory="my_fashion_mnist",project_name="hyperband")

In [50]:
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(X_train, y_train, epochs=10,
validation_data=(X_val, y_val),
callbacks=[early_stopping_cb,
tensorboard_cb])

Trial 60 Complete [00h 00m 40s]
val_accuracy: 0.6655036211013794

Best val_accuracy So Far: 0.7405828237533569
Total elapsed time: 00h 20m 27s
INFO:tensorflow:Oracle triggered exit


linux

1- با چه دستوری مسیر پایتون را پیدا کنیم

whereis python

whereis python3

all filees python:


find / -type f | while read in ; do if file -i "${in}" | grep -q x-python ; then echo "${in}" ; fi ; done

2- با چه دستوری از نوع فایل مطمین میشبم

Displaying the file type (file command)


To display the type of information the file named myfile contains, type the following: file myfile. ...


To display the type of each file named in the filenames.lst file, which contains a list of file names, type the following: file -f filenames.lst.

3-منظور از اعداد نشان داده شده بعد از زدت دستور زیر چیست

The numbered files are directories that correspond to process numbers or process IDs (PIDs)

4- دستوری که دایرکتوری بسازد که درون آن یک دایرکتوری دیگر هم هست

With the combination of both mkdir and touch commands, we can accomplish the task of creating a directory and a file in a single go. Here, the parent option of mkdir helps us to create the parent directory (unless it exists) without any error, while the touch command creates a file.
mkdir -p

5- با چه دستوری فایل بالا را به صورت ریکرسیو پاک می کنید

To remove a directory and all its contents, including any subdirectories and files, use the rm command with the recursive option, -r . Directories that are removed with the rmdir command cannot be recovered, nor can directories and their contents removed with the rm -r command.

6- با چه کامتد هایی میتوان فایل را خواند 

cat command – Display text file.


less command – Show text file one screen at a time.


more command – Same as less command


gnome-open command or xdg-open command (generic version) or kde-open command (kde version) – Linux gnome/kde desktop command to open any file.


open command – macOS (OS X) Unix specific command to open any file.


file command – Determine file type.


ls command – List directory contents on Linux or Unix.


Files or Dolphin GUI file managers


mc command – Visual shell for Unix-like systems



7- برای تعقیب تعییرات یک فایل چه دستوری را پیشنهاد میدهید

stat tells you various information about the file including the last modified date.

Access shows the time of last data access (e.g. read).
Modify shows the time of last data modification.
Change shows the time the file status last changed.

Another way that is more flexible is using date -r. From man date
This has the advantage of allowing you to specify the output format

8- معادل دستورپرینت در بش چیست 

CUPS is the printing system under Linux. In particular, it provides a server and clients. The lp and lpr are two common commands to print files: lpr is the BSD one, and lp the System V one. There exist various implementations (more or less compatible with the original commands), but nowadays they should be CUPS clients.
To print files from Linux terminal, execute the lp and lpr command. It will print the file from the default printer. Execute it as follows:



lp < file name>  
lpr <file name>  
The above commands will print the specified file.